# Разработка классификатора новостей

**Нужно:**
* выбрать какой-либо новостной ресурс, где к новостям привязаны категории или метки (например http://lenta.ru, http://fontanka.ru, http://gazeta.ru)
* загрузить новости по некоторому набору (5-10) категорий за пару лет
* обучить классификатор на эти новостях
* продемонстрировать его работу, разработав простеший web-интерфейс (вариант - telegram-бот), куда пользователь вводит текст новости и на выходе получает наиболее вероятную категорию. В качестве фреймворка проще всего взять [Flask](http://flask.pocoo.org) (см. примеры) .

In [2]:
import lxml.html
import lxml.etree
import requests
import sqlite3
from tqdm import tqdm, tqdm_notebook

import pickle

In [4]:
conn = sqlite3.connect("news.db") 
cursor = conn.cursor()
 
# Создание таблицы
#cursor.execute("""CREATE TABLE News
#                 ( category text, date text, title text, article text)
#             """)


In [3]:

path = 'http://m.lenta.ru'
response= requests.get(path, verify=False)
response.status_code


tree = lxml.html.fromstring(response.text)

C:\Users\Nuts\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [5]:
#даты 
month_31 = ['01','03','05','07','08','10','12']#не забыть вернуть январь
month_30 = ['04','06','09','11']
month_29 = ['02']

day_31=[]
day_30= []
day_10 = ['01','02','03','04','05','06','07','08','09']


for k in range(10,31,1):
    str_k = str(k)
    day_30.append(str_k)
    
day_30 = day_10 + day_30 

for k in range(10,32,1):
    str_k = str(k)   
    day_31.append(str_k)
    
    
day_31 = day_10 + day_31 


    

    
date = []

for year in range(2017,2018,1):
    for month in month_30:
        for day in day_30:
            d = '/%d/%s/%s'%(year,month,day)
            date.append(d)
            
    for month in month_31:
        for day in day_31:
            d = '/%d/%s/%s'%(year,month,day)
            date.append(d)
 
            
links = []        
for d in date :        
    links.append(path + d)

#по датам
    

In [8]:
for link in tqdm_notebook(links):

        response= requests.get(link, verify=False)
        tree = lxml.html.fromstring(response.text)
         статьи
        day_links = tree.xpath('//div[@class="item news b-tabloid__topic_news"]/div[@class = "titles"]//a/@href')
        i = -1
      #  for que in tree.xpath('//div[@class="item news b-tabloid__topic_news"]/div[@class = "titles"]/h3'):
       #     i=i+1
        #    quest = que.xpath('a/@href')
            
         #   way = path + quest.pop()
          #  resp= requests.get(way, verify=False)
           # tree_day = lxml.html.fromstring(resp.text)
            #category = tree_day.xpath('//div[@class="b-subheader__title js-nav-opener"]/text()')
            #article = tree_day.xpath('//div[@class="b-topic__body"]/p//text()')
            #print(category)
            #title = tree_day.xpath('//h1[@class="b-topic__title"]/text()')
            #date = tree_day.xpath('//div[@class="b-topic__date"]/text()')
            #cursor.execute("INSERT INTO News VALUES (?,?,?,?)", (category[0], date[0], title[0], article[0]))
            #conn.commit()
            
       

In [3]:
sql = "SELECT count(*) FROM News"
cursor.execute(sql)
print(cursor.fetchall())
   

[(22898,)]


In [4]:
sql = "SELECT count(*) FROM News where category = ? "
cursor.execute(sql,['Из жизни'])
print( cursor.fetchall())

[(0,)]


In [59]:

sql = "Delete  FROM News where category = ? "
cursor.execute(sql,['Из жизни'])
conn.commit()
print( cursor.fetchall())

[]


In [60]:
sql = "SELECT DISTINCT category FROM News "
cursor.execute(sql)
print( cursor.fetchall())

[('Наука и техника',), ('Интернет и СМИ',), ('Бывший СССР',), ('Россия',), ('Спорт',), ('Мир',), ('Культура',), ('Силовые структуры',), ('Финансы',)]


In [5]:
from sklearn import svm
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer






In [6]:
conn = sqlite3.connect("news.db") 
cursor = conn.cursor()


sql = "Select  category FROM News "
cursor.execute(sql)
targets = [t[0] for t in cursor.fetchall()[:22000]] 
    
    
sql = "Select article FROM News "
cursor.execute(sql)
text = [t[0] for t in cursor.fetchall()[:22000]]
     
    
count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()
 
X = count_vect.fit_transform(text)
X = tfidf_transformer.fit_transform(X)






X_train, X_test, y_train, y_test = train_test_split(X, targets, test_size=0.3, random_state=0)
#clf = LogisticRegression(C=100).fit(X_train, y_train)
clf = svm.SVC(kernel='linear', C=100).fit(X_train, y_train)
clf.score(X_test, y_test) 



                            



0.80590909090909091

In [34]:

predicted_targets=[]
sql = "Select article FROM News "
cursor.execute(sql)
for line in cursor.fetchall()[15001:15115]:
        predicted = clf.predict(count_vect.transform(line).toarray())
        predicted_targets.append(predicted[0])


sql = "Select  category FROM News "
cursor.execute(sql)

select_targets = [t[0] for t in cursor.fetchall()[15001:15115]] 

In [7]:

with open('count_vect.pkl', 'wb') as f:
    pickle.dump(count_vect,f)


with open('clf_svm.pkl', 'wb') as f: 
    pickle.dump(clf, f) 


In [3]:
with open('clf_svm.pkl', 'rb') as f: 
    clf = pickle.load(f) 
    
with open('count_vect.pkl', 'rb') as f: 
    count_vect = pickle.load(f) 

In [5]:
que = "Допинг в российском биатлоне"
predicted = clf.predict(count_vect.transform([que]).toarray())
str(predicted[0])

'Спорт'